In [16]:
import pandas as pd
import numpy as np
import re
import time

from datetime import datetime as dt

### Preprocessing

In [101]:
data = pd.read_csv("bank_transactions.csv")

In [102]:
data.dropna(inplace=True)

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 79.5+ MB


In [104]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


### I don't need all the columns in this datam

In [105]:
def cleaning(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [106]:
data.TransactionTime = data.TransactionTime.apply(lambda x: dt.strptime(str(x).zfill(6), '%H%M%S').hour)

In [107]:
data.CustomerDOB = pd.to_datetime(data.CustomerDOB).dt.to_period('M')

In [108]:
data.TransactionDate = pd.to_datetime(data.TransactionDate).dt.to_period('M')

In [109]:
data = data.iloc[:, 2:]

## Place the numeric variable in a range of values

In [110]:
data.rename(columns = {"TransactionAmount (INR)": "TransactionAmount_INR"}, inplace = True)

In [147]:
n1 = data.CustAccountBalance.max()/100
n2 = data.TransactionAmount_INR.max()/100

In [148]:
range1 = range(int(data.CustAccountBalance.max()/100), int(data.CustAccountBalance.max())+int(n1), int(data.CustAccountBalance.max()/100))
range2 = range(int(data.TransactionAmount_INR.max()/100), int(data.TransactionAmount_INR.max())+int(n2), int(data.TransactionAmount_INR.max()/100))

In [134]:

data.CustAccountBalance = data.CustAccountBalance.apply(lambda x :  range1[int(x/n1)])

In [149]:
data.TransactionAmount_INR = data.TransactionAmount_INR.apply(lambda x :  range1[int(x/n2)])

In [150]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR
0,1994-10,F,JAMSHEDPUR,1150354,2016-02,14,1161857
1,2057-04,M,JHAJJAR,1150354,2016-02,14,2323714
2,1996-11,F,MUMBAI,1150354,2016-02,14,1161857
3,1973-09,F,MUMBAI,1150354,2016-02,14,1161857
4,1988-03,F,NAVI MUMBAI,1150354,2016-02,18,1161857


let's create a new column with the joined string of all previous columns

In [151]:
data['joined_data'] = data[data.columns[1:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)
#Time that it took: 
#around 1m

In [152]:
#Clean the data that i'm going to use
data["joined_data"] = data["joined_data"].apply(lambda x: " ".join(cleaning(x)))


In [153]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR,joined_data
0,1994-10,F,JAMSHEDPUR,1150354,2016-02,14,1161857,f jamshedpur 1150354 201602 14 1161857
1,2057-04,M,JHAJJAR,1150354,2016-02,14,2323714,m jhajjar 1150354 201602 14 2323714
2,1996-11,F,MUMBAI,1150354,2016-02,14,1161857,f mumbai 1150354 201602 14 1161857
3,1973-09,F,MUMBAI,1150354,2016-02,14,1161857,f mumbai 1150354 201602 14 1161857
4,1988-03,F,NAVI MUMBAI,1150354,2016-02,18,1161857,f navi mumbai 1150354 201602 18 1161857


In [154]:
prova = data.joined_data.tolist()
prova2 = set()
for x  in prova:
    for y in x.split():
        prova2.add(y)


In [155]:
len(prova2)

6718

In [172]:
from sklearn.feature_extraction.text import CountVectorizer

In [177]:
vectorizer = CountVectorizer( analyzer='word',  ngram_range=(1,1))
prova =vectorizer.fit_transform(data["joined_data"])
# df = pd.DataFrame(prova, columns = vectorizer.get_feature_names_out())
prova.toarray()

MemoryError: Unable to allocate 51.9 GiB for an array with shape (1041614, 6685) and data type int64

In [170]:
x =vectorizer.inverse_transform(prova)
df = pd.DataFrame(x, columns = vectorizer.get_feature_names_out())

AttributeError: 'list' object has no attribute 'todense'

In [ ]:
df.head()